In [77]:
# Bibiotecas
import geopandas as gpd
import matplotlib.pyplot as plt
import contextily as cx
import shapely.geometry 
import shapely.wkt 
import folium as fl
import pandas as pd
from functools import partial
import pyproj

In [78]:
# Abre o dataframe corretamente
FILE_PARADAS = "../../../databases/paradas/paradas.shp"
FILE_LINHAS = "../../../databases/linhas/Linhas.shp"

from shapely.geometry import Point
import pyproj

def converter_para_EPSG_4326(point):
    # Define a transformação de EPSG 31983 para EPSG 4326
    transformer = pyproj.Transformer.from_crs("epsg:31983", "epsg:4326", always_xy=True)

    # Converte o ponto
    lon, lat = transformer.transform(point.x, point.y)
    ponto_EPSG_4326 = Point(lon, lat)

    return ponto_EPSG_4326


In [79]:
from shapely.geometry import Point
from shapely.ops import transform
from functools import partial
import pyproj

def calcular_distancia_km(point1, point2):
    project_to_EPSG_4326 = partial(
        pyproj.transform,
        pyproj.Proj(init='epsg:31983'),  # Projeto de origem (EPSG 31983)
        pyproj.Proj(init='epsg:4326'),  # Projeto de destino (EPSG 4326)
    )

    # Transforma os pontos para o EPSG 4326
    point1_wgs84 = transform(project_to_EPSG_4326, point1)
    point2_wgs84 = transform(project_to_EPSG_4326, point2)

    # Calcula a distância entre os pontos em graus
    distancia_graus = point1_wgs84.distance(point2_wgs84)

    # Converte a distância para metros (considerando que 1 grau de latitude ~= 111.32 km)
    distancia_metros = distancia_graus * 111320.0

    return distancia_metros

In [80]:
# Armazena as distancias de uma linha
def Armazenar_distancias(df, distancias):
    for i in range(len(df)):
        if (i != len(df)):
            distancia = calcular_distancia_km(df.iloc[i], df.iloc[i+1])
            distancias.append(distancia)

# Calcula a media a partir da funcao Armazenar_distancias
def calcular_media(distancias):
    if len(distancias) == 0:
        return 0
    soma = sum(distancias)
    media = soma / len(distancias)
    return media

# Calcula a moda a partir da função Armazenar_distancias
def calcular_moda(distancias):
    moda = distancias.mode
    return moda

In [81]:
# Teste da função da media
df = pd.DataFrame({'Latitude': [40.7128, 34.0522, 41.8781, 29.7604],
                   'Longitude': [-74.0060, -118.2437, -87.6298, -95.3698]})
distancias = []
Armazenar_distancias(df,distancias)
media = calcular_media(distancias)

/home/jrxpinheiro/.local/lib/python3.10/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/home/jrxpinheiro/.local/lib/python3.10/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


AttributeError: 'Series' object has no attribute 'is_empty'

In [ ]:
# Remover paradas usnado limiar e deixar dataframe das paradas em ordem
# Uma ideia de como calcular o limiar
def limiar (media):
    media = media * 0,8
    return media

In [ ]:
# Plotar uma rota de paradas.